### Before submission, check box areas are not zero

In [5]:

import cv2
import numpy as np
import pandas as pd

df = pd.read_csv('../submit_zips/oriented_rcnn_r101_fpn_1x_dota_ms_with_flip_rotate_balance_cate.csv',header=None) # s2anet_r50_fpn_1x_fair1m_1_5,  oriented_rcnn_r101_fpn_1x_dota_ms_with_flip_rotate_balance_cate
df = df.round(3)
ct = 0
temp_name = ''
to_drop = []
print(df.shape[0])
for i in range(df.shape[0]):
    x1,y1 = df.iloc[i,2],df.iloc[i,3]
    x2,y2 = df.iloc[i,4],df.iloc[i,5]
    x3,y3 = df.iloc[i,6],df.iloc[i,7]
    x4,y4 = df.iloc[i,8],df.iloc[i,9]
    area = np.sqrt((x1-x2)**2+(y1-y2)**2) * np.sqrt((x3-x2)**2+(y3-y2)**2)
    if area == 0:
        to_drop.append(i)
        # print(i)

droped_df = df.drop(to_drop)
print(droped_df.shape[0], len(to_drop))
droped_df.to_csv('postpocessed.csv',index=False,header=False)

27315
27315 0


### Draw BBox on test images

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
if not os.path.exists('../../res_with_box/'):
    os.mkdir('../../res_with_box/')

if not os.path.exists('../../res_test_gt_vis/'):
    os.mkdir('../../res_test_gt_vis/')
# gt_color = (0, 0, 0)

# gts = np.load('../gts.npy')
# gt_df = pd.DataFrame(gts)
# gt_df = gt_df.astype({0: int, 1: int, 2: int,3: int,4: int,5: int,6: int,7: int,8: int})

# file_names = set(gt_df[0])


# # # read image
# # ct = 0

# for img_name in file_names:
#     result = cv2.imread('/opt/data/private/LYX/data/data/test/images/'+str(img_name)+'.tif')
#     # print(result)
#     gt_img_labels = gt_df[gt_df[0]==img_name].loc[:,1:8]

#     for index, row in gt_img_labels.iterrows():
     
   
#         x1,y1 = row[1],row[2]
#         x2,y2 = row[3],row[4]
#         x3,y3 = row[5],row[6]
#         x4,y4 = row[7],row[8]

#         cv2.line(result, (x1, y1), (x2, y2), gt_color, 1)
#         cv2.line(result, (x2, y2), (x3, y3), gt_color, 1)
#         cv2.line(result, (x3, y3), (x4, y4), gt_color, 1)
#         cv2.line(result, (x4, y4), (x1, y1), gt_color, 1)

#     cv2.imwrite('../../res_test_gt_vis/'+str(img_name)+'.jpg',result) 


rgb_dict={'Airplane':(70, 240, 240), 'Ship':(0, 0, 128), 'Vehicle':(128, 128, 0), 'Basketball_Court':(255,225,25), 'Tennis_Court':(210, 245, 60), 'Football_Field':(245, 130, 48), 'Baseball_Field':(255, 215, 180), 'Intersection':(60, 180, 75), 'Roundabout':(145, 30, 180), 'Bridge':(220, 190, 255)}

df = pd.read_csv('../../submit_zips/orcnn_r152_fpn_ms_flip_rotate_bc_fair_dota_epoch12.csv',header=None) # s2anet_r50_fpn_1x_fair1m_1_5,  oriented_rcnn_r101_fpn_1x_dota_ms_with_flip_rotate_balance_cate
df = df.astype({2: int,3: int,4: int,5: int,6: int,7: int,8: int,9: int})
# read image
ct = 0
temp_name = ''
for i in range(df.shape[0]):
    if df.iloc[i,-1]<0.5:
        continue
    img_name = df.iloc[i,0]
    if temp_name!=img_name:
        if len(temp_name)>0:
            cv2.imwrite('../../res_with_box/'+img_name[:-3]+'jpg',result) 
        temp_name = img_name
        
        result = cv2.imread('../../res_test_gt_vis/'+img_name[:-3]+'jpg')
    cls_name = df.iloc[i,1]
    cfdc = df.iloc[i,-1]
    x1,y1 = df.iloc[i,2],df.iloc[i,3]
    x2,y2 = df.iloc[i,4],df.iloc[i,5]
    x3,y3 = df.iloc[i,6],df.iloc[i,7]
    x4,y4 = df.iloc[i,8],df.iloc[i,9]

    colr = rgb_dict[cls_name]

    # cv2.line(image, start_point, end_point, color, thickness)
    cv2.line(result, (x1, y1), (x2, y2), colr, 1)
    cv2.line(result, (x2, y2), (x3, y3), colr, 1)
    cv2.line(result, (x3, y3), (x4, y4), colr, 1)
    cv2.line(result, (x4, y4), (x1, y1), colr, 1)
    # cv2.putText(result,cls_name+' '+str(cfdc),(x1, y1),0,0.4,colr) #cv2.putText(image, 'OpenCV', org, font, fontScale, color, thickness, cv2.LINE_AA)

    
    # save resulting image
    # cv2.imwrite('../../res_with_box/'+img_name[:-3]+'jpg',result)     
    ct+=1
    # if ct==10:break
# cv2.imshow('window_name', result) 
# result

#### Preprocess FAIR1M2.0 data, remove and rename classes and move files to target folders

In [23]:
# Bridge Roundabout Intersection Baseball_Field Football_Field Tennis_Court Basketball_Court Vehicle Ship Airplane

import os
import os.path as osp
import xml.etree.ElementTree as ET
import pprint
import shutil
pp = pprint.PrettyPrinter(depth=4)

jdet_coarse2fine_mappings = {
    "Airplane": [
        "A220", "A321", "A330", "A350", "ARJ21", "Boeing737", "Boeing747",
        "Boeing777", "Boeing787", "C919", "other-airplane"],
    "Ship": [
        "Tugboat", "other-ship", "Liquid Cargo Ship", "Motorboat",
        "Passenger Ship", "Dry Cargo Ship", "Warship", "Engineering Ship",
        "Fishing Boat"],
    "Vehicle": [
        "other-vehicle", "Bus", "Cargo Truck", "Small Car", "Dump Truck",
        "Van", "Excavator", "Tractor", "Trailer", "Truck Tractor"],
    "Basketball_Court": ["Basketball Court"],
    "Tennis_Court": ["Tennis Court"],
    "Football_Field": ["Football Field"],
    "Baseball_Field": ["Baseball Field"],
    "Intersection": ["Intersection"],
    "Roundabout": ["Roundabout"],
    "Bridge": ["Bridge"],
}


data_xml_root = './' # "/media/data3/lyx/Detection/data/train/labelXml/"
data_img_root = './' # "/media/data3/lyx/Detection/data/train/images/"
target_xml_root =  'out' #  "/media/data3/lyx/Detection/merge_data/train/labelXml/"
target_img_root =  'out' #  "/media/data3/lyx/Detection/merge_data/train/images/"

distributions = dict()
total_imgs = 0

sub_dir_path = data_xml_root
xml_files = [f for f in os.listdir(sub_dir_path)
                if osp.isfile(osp.join(sub_dir_path, f))]
# print(xml_files)
for xml_file in xml_files:
    print(xml_file)
    if not xml_file.endswith('.xml'):
        continue
    xml_path = osp.join(sub_dir_path, xml_file)
    tree = ET.parse(xml_path)

    root = tree.getroot()
    if len(root.findall('objects')) != 1:
        raise ValueError('xml file {} has more than one objects'.format(xml_file))
    objects = root.find('objects')
    for obj in objects.findall('object'):
        possible_result = obj.find('possibleresult')
        cls_name = possible_result.find('name').text

        # rename object
        found = False
        for key, value in jdet_coarse2fine_mappings.items():
            if key == cls_name:
                found = True
            else:
                for v in value:
                    if v == cls_name:
                        possible_result.find('name').text = key
                        found = True
            
        # remove object
        if not found:
            objects.remove(obj)
        
    if len(objects)>0:
        # print('save')
        # tree.write('out.xml')
        png_file = xml_file.split('.')[0]+'png'
        # tree.write(osp.join(target_xml_root + xml_file))
        # shutil.copyfile(data_img_root+png_file, target_img_root + png_file) 


    break

3290.xml
save


### Get primary train data information and statistics   

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import glob
import numpy as np
root = '/media/data3/lyx/Detection/data/train/labelXml'
files = sorted(glob.glob(root+ "/*.*"))
cls_name = []
file_name = []
x1=[]
y1=[]
x2=[]
y2=[]
x3=[]
y3=[]
x4=[]
y4=[]
pts_list = []

for each_file in files:
    with open(each_file, 'r') as f:
        data = f.read()
    
    # Passing the stored data inside
    # the beautifulsoup parser, storing
    # the returned object
    Bs_data = BeautifulSoup(data, "xml")
    
    # Finding all instances of tag
    # `unique`
    objects = Bs_data.find_all('object')
    
    objects[0].find_all('point')[0].text

    for i in range(len(objects)):
        obj = objects[i]
        file_name.append(each_file.split('/')[-1])
        cls_name.append(obj.find('name').text)
        points = obj.find_all('point')
        pts = []
        for each in points:
            pts.append(eval(each.text)[0])
            pts.append(eval(each.text)[1])
        pts_list.append(pts)

pts_list = np.array(pts_list)
x1=pts_list[:,0]
y1=pts_list[:,1]
x2=pts_list[:,2]
y2=pts_list[:,3]
x3=pts_list[:,4]
y3=pts_list[:,5]
x4=pts_list[:,6]
y4=pts_list[:,7]

data = {'name':cls_name,'file':file_name,  'x1':x1,'y1':y1,'x2':x2,'y2':y2,'x3':x3,'y3':y3,'x4':x4,'y4':y4}
df = pd.DataFrame(data)
df.to_csv('train_set_data.csv',index=False)

df['area'] = np.sqrt((df['x1']-df['x2'])**2+(df['y1']-df['y2'])**2) * np.sqrt((df['x3']-df['x2'])**2+(df['y3']-df['y2'])**2)
width = np.sqrt((df['x1']-df['x2'])**2+(df['y1']-df['y2'])**2)
length = np.sqrt((df['x3']-df['x2'])**2+(df['y3']-df['y2'])**2)
for i in range(len(width)):
    if width[i] > length[i]:
        l = width[i]
        width[i] = length[i]
        length[i] = l
df['width'] = width
df['length'] = length
df['ratio'] = length/width

# normed_area
cls_names = []
stats = []
for n in ['Airplane','Ship', 'Vehicle', 'Basketball Court', 'Tennis Court', 'Football Field', 'Baseball Field', 'Intersection', 'Roundabout', 'Bridge']:
    cls_names.append(n)
    areas = df[df['name']==n]['area']
    mean_area = areas.mean()
    normed_var_area= np.var(areas/mean_area)
    ratios = df[df['name']==n]['ratio']
    mean_ratios= ratios.mean()
    normed_var_ratios= np.var(ratios/mean_ratios)
    stats.append([len(areas), mean_area, normed_var_area,mean_ratios, normed_var_ratios])
stats = np.array(stats)
stats_dict = {'name':cls_names, 'amount':stats[:,0],'mean_area':stats[:,1],'normed_var_area':stats[:,2],'mean_ratio':stats[:,3],'normed_var_ratio':stats[:,4]}
stats_df = pd.DataFrame(stats_dict)
stats_df.to_csv('train_set_stats.csv',index=False)



### Get processed train data information and statistics (Please refer to this statistics)

please delete image and txt file of P5877_1.0_976_976 and P5877_1.0_824_976 becase the processed bbox is actually a line without area

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import glob
import numpy as np
root = '/media/data3/lyx/Detection/preprocessed_ms/train_1024_200_0.5-1.0-1.5/labelTxt/'
# root = '/media/data3/lyx/Detection/preprocessed/train_1024_200_1.0/labelTxt/'
files = sorted(glob.glob(root+ "/*.*"))
cls_name = []
file_name = []
x1=[]
y1=[]
x2=[]
y2=[]
x3=[]
y3=[]
x4=[]
y4=[]
pts_list = []

for each_file in files:
    file1 = open(each_file, 'r')
    Lines = file1.readlines()
    for line in Lines:
        file_name.append(each_file.split('/')[-1])
        pts = []
        l =line.strip().split(' ')
        cls_name.append(l[8])
        for i in range(8):
            pts.append(eval(l[i]))
        pts_list.append(pts)


pts_list = np.array(pts_list)
x1=pts_list[:,0]
y1=pts_list[:,1]
x2=pts_list[:,2]
y2=pts_list[:,3]
x3=pts_list[:,4]
y3=pts_list[:,5]
x4=pts_list[:,6]
y4=pts_list[:,7]

data = {'name':cls_name,'file':file_name,  'x1':x1,'y1':y1,'x2':x2,'y2':y2,'x3':x3,'y3':y3,'x4':x4,'y4':y4}
df = pd.DataFrame(data)

df['area'] = np.sqrt((df['x1']-df['x2'])**2+(df['y1']-df['y2'])**2) * np.sqrt((df['x3']-df['x2'])**2+(df['y3']-df['y2'])**2)
width = np.sqrt((df['x1']-df['x2'])**2+(df['y1']-df['y2'])**2)
length = np.sqrt((df['x3']-df['x2'])**2+(df['y3']-df['y2'])**2)
for i in range(len(width)):
    if width[i] > length[i]:
        l = width[i]
        width[i] = length[i]
        length[i] = l
df['width'] = width
df['length'] = length
df['ratio'] = length/width
thin_box = df.loc[df['area'] == 0]
print(thin_box)
df.to_csv('train_set_data.csv',index=False)


cls_names = []
stats = []
for n in ['Airplane','Ship', 'Vehicle', 'Basketball_Court', 'Tennis_Court', 'Football_Field', 'Baseball_Field', 'Intersection', 'Roundabout', 'Bridge']:
    cls_names.append(n)
    areas = df[df['name']==n]['area']
    mean_area = areas.mean()
    normed_var_area= np.var(areas/mean_area)
    ratios = df[df['name']==n]['ratio']
    mean_ratios= ratios.mean()
    normed_var_ratios= np.var(ratios/mean_ratios)
    stats.append([len(areas), mean_area, normed_var_area,mean_ratios, normed_var_ratios])
stats = np.array(stats)
stats_dict = {'name':cls_names, 'amount':stats[:,0],'mean_area':stats[:,1],'normed_var_area':stats[:,2],'mean_ratio':stats[:,3],'normed_var_ratio':stats[:,4]}
stats_df = pd.DataFrame(stats_dict)
stats_df.to_csv('train_set_stats.csv',index=False)



Empty DataFrame
Columns: [name, file, x1, y1, x2, y2, x3, y3, x4, y4, area, width, length, ratio]
Index: []


In [7]:
from cmath import inf


len(df)
thin_box = df.loc[df['area'] == 0]
thin_box.to_csv('thin_box.csv',index=False)